In [21]:
import pandas as pd
import numpy as np

In [22]:
gameToPlayerDf = pd.read_csv("gameToPlayer.csv")
gameDf = pd.read_csv("Kaggle Data/game.csv")

In [23]:
gameDf[gameDf["game_date"] == gameDf["game_date"].min()]

,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,...,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,video_available_away,season_type
0,21946,1610610035,HUS,Toronto Huskies,24600001,1946-11-01 00:00:00,HUS vs. NYK,L,0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,68.0,2,0,Regular Season


In [24]:
# Group players based on team id and game id so that we get individual teams and their players
grouped_df = gameToPlayerDf.groupby(['game_id','team_id'])['player_id'].apply(list).reset_index()
grouped_df = grouped_df.sort_values(by="game_id")
grouped_df.sort_values(by="game_id")

,game_id,team_id,player_id
0,11300001,FBU,"[1962936724, 42541, 42531, 42547, 42540, 42546..."
1,11300001,OKC,"[201142, 201586, 2570, 965, 200757, 203500, 25..."
2,11300002,CHI,"[2430, 203513, 2399, 203104, 2550, 202710, 173..."
3,11300002,IND,"[101122, 101133, 2561, 201228, 201579, 202362,..."
4,11300003,HOU,"[203496, 202718, 2730, 201976, 201935, 201166,..."
...,...,...,...
59631,49800085,SAS,"[1495, 1562, 251, 990, 1114, 422, 53, 70, 764,..."
59632,49800086,NYK,"[369, 948, 275, 201, 703, 164, 84, 913]"
59633,49800086,SAS,"[1495, 251, 990, 760, 1114, 422, 53, 70, 764]"
59634,49800087,NYK,"[369, 948, 275, 201, 703, 1006, 164, 84, 913]"


In [25]:
grouped_df[grouped_df["game_id"] == 24600001]

,game_id,team_id,player_id


In [26]:
# Adding winning team name in the game DF
gameDf['winning_team_abbr'] = gameDf.apply(lambda row: row['team_abbreviation_home'] if row['wl_home'] == 'W' else row['team_abbreviation_away'], axis=1)
teamWinDf = gameDf[["game_id","winning_team_abbr"]]
teamWinDf.sort_values(by="game_id")
teamWinDf

,game_id,winning_team_abbr
0,24600001,NYK
1,24600003,BOM
2,24600002,PRO
3,24600004,CHS
4,24600005,WAS
...,...,...
65693,42200403,DEN
65694,42200404,DEN
65695,42200405,DEN
65696,32200001,GNS


In [27]:
teams = pd.read_csv("Kaggle data/team.csv")
teamWinDf = pd.merge(teamWinDf, teams, left_on='winning_team_abbr', right_on='abbreviation', how='left')[['game_id', 'winning_team_abbr', 'nickname']]
teamWinDf

,game_id,winning_team_abbr,nickname
0,24600001,NYK,Knicks
1,24600003,BOM,NaN
2,24600002,PRO,NaN
3,24600004,CHS,NaN
4,24600005,WAS,Wizards
...,...,...,...
65693,42200403,DEN,Nuggets
65694,42200404,DEN,Nuggets
65695,42200405,DEN,Nuggets
65696,32200001,GNS,NaN


In [28]:
# Append winning team name in the grouped_df object
teamsWithWinDf = teamWinDf.merge(grouped_df, on="game_id")
teamsWithWinDf

,game_id,winning_team_abbr,nickname,team_id,player_id
0,29600012,LAL,Lakers,LAL,"[76, 916, 224, 89, 965, 2, 0, 922, 399, 406, 9..."
1,29600012,LAL,Lakers,PHX,"[208, 330, 959, 0, 364, 47, 445, 109, 714, 170..."
2,29600005,MIA,Heat,ATL,"[363, 1096, 80, 0, 468, 695, 302, 87, 895, 120]"
3,29600005,MIA,Heat,MIA,"[0, 703, 390, 105, 896, 173, 1134, 297, 202, 136]"
4,29600002,CLE,Cavaliers,NJN,"[355, 383, 420, 0, 762, 123, 1365, 234, 709, 9..."
...,...,...,...,...,...
59661,42200404,DEN,Nuggets,MIA,"[1629312, 203469, 200768, 201567, 1628389, 162..."
59662,32200001,GNS,NaN,GNS,"[1629630, 1628369, 1628983, 201950, 203081, 20..."
59663,32200001,GNS,NaN,LBN,"[1627759, 203954, 203944, 1630169, 203999, 163..."
59664,32200001,GNS,NaN,GNS,"[1629630, 1628369, 1628983, 201950, 203081, 20..."


In [29]:
teamsWithWinDf.to_csv("teamsWithWin.csv")